# Create a Forecast plot given a stored model

### Notebook Configuration

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

# logging.basicConfig(level=logging.INFO)

### Imports

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["axes.axisbelow"] = True
import matplotlib.dates as mdates
import numpy as np
import pandas
import pandas as pd
from pandas import DataFrame, Index, Series, Timedelta, Timestamp

rng = np.random.default_rng()
np.set_printoptions()

from copy import deepcopy

import torch
import torchinfo
from torch import Tensor, jit, tensor
from torch.utils.data import DataLoader

### Enter Path and Name

In [ ]:
ID = (449, 15653)  # the Run_id / Experiment_id we want to plot.
# files:  2021-11-12T00:52:10 2021-11-12T00:51:55 2021-11-12T00:51:48
# "checkpoints/2021-11-15T12:05:00/LinODEnet-0"
# "checkpoints/LinODEnet/KIWI_RUNS/skew_allways/2021-11-15T16:05:41/LinODEnet-0"
# "adam/2021-11-15T20:38:52/LinODEnet-0"
PATH = "/home/rscholz/Projects/KIWI/tsdm/experiments/kiwi/checkpoints/KIWI/LinODEnet/2022-11-16T01:55:32/f=0_bs=64_lr=0.001_hs=60_ls=128/"
NAME = "RecursiveScriptModule-24-3168"
# the model checkpoint, should be a zip-archive created by torch.save / torch.jit.save
MODEL_FILE = PATH + NAME
DEVICE = torch.device("cpu")
DTYPE = torch.float32
NAN = tensor(float("nan"), dtype=DTYPE, device=DEVICE)
PRD_HORIZON = "1h"
OBS_HORIZON = "2h"
# HORIZON = "3h"

## Intialize the task

In [ ]:
from tsdm.tasks import KiwiTask

task = KiwiTask(
    sampler_kwargs=dict(
        forecasting_horizon=PRD_HORIZON,
        observation_horizon=OBS_HORIZON,
    )
)

## Load the Model

In [ ]:
model = torch.jit.load(MODEL_FILE, torch.device("cpu"))
torchinfo.summary(model, depth=1)

## Import Task

In [ ]:
# assert ID in task.splits[(0, "train")][0].index
TRAINLOADER = task.dataloaders[(0, "train")]
EVALLOADER = task.dataloaders[(0, "test")]

ts = task.dataset[ID].timeseries

dloader = TRAINLOADER
split = task.splits[(0, "test")][ID]
encoder = task.encoders[(0, "test")]
sampler = task.samplers[(0, "test")][ID]
sampler.shuffle = False

from tsdm.tasks import TimeSeriesSampleGenerator

generator = TimeSeriesSampleGenerator(
    split,
    observables=task.observables,
    targets=task.targets,
    covariates=task.covariates,
)

samples = [generator[key] for key in sampler];

In [ ]:
sample = samples[0]
tx, x = encoder.encode(sample.inputs.x).values()
tx, x = encoder.encode(sample.inputs.x).values()
ty, y = encoder.encode(sample.targets.y).values()
yhat = model(tx, x)
reconstructed = encoder.decode({"T": tx, "X": yhat})

In [ ]:
reconstructed.index  # .loc[: task.observation_horizon]

### Helper Function for Batch post-processing

In [ ]:
def prep_batch(batch: tuple[Tensor, Tensor]):
    """Get batch and create model inputs and targets."""
    T, X = batch
    targets = X[..., task.observation_horizon :, task.targets.index].clone()
    # assert targets.shape == (BATCH_SIZE, PRD_HORIZON, len(TASK.targets))
    originals = X.clone()
    inputs = X.clone()
    inputs[:, task.observation_horizon :, task.targets.index] = NAN
    inputs[:, task.observation_horizon :, task.observables.index] = NAN
    # assert inputs.shape == (BATCH_SIZE, HORIZON, NUM_DIM)
    return T, inputs, targets, originals

## Helper function to create the plot

In [ ]:
def make_plot(axes, task, sample):
    """Helper function to create plot automatically."""
    # batch = slices[0]
    tx, x = encoder.encode(sample.inputs.x).values()
    ty, y = encoder.encode(sample.targets.y).values()
    yhat = model(tx, x)
    reconstructed = encoder.decode({"T": tx, "X": yhat})
    # mask = sample.inputs.x.notna() | sample.targets.y.notna()

    time = reconstructed.index.to_series()
    t0 = time[0]
    t1 = t0 + pd.Timedelta(task.observation_horizon)
    t2 = t1 + pd.Timedelta(task.forecasting_horizon)

    # # convert to float
    # time = time / np.timedelta64(1, "h")
    # t0 = t0 / np.timedelta64(1, "h")
    # t1 = t1 / np.timedelta64(1, "h")
    # t2 = t2 / np.timedelta64(1, "h")
    # reconstructed.index = time
    # time = time.to_series()
    # display(t0, t1,  time.loc[t0:t1])

    for ax, target in zip(axes.flatten(), task.targets):
        # color = next(ax._get_lines.prop_cycler)["color"]
        ax.axvspan(t0.to_numpy(), t1.to_numpy(), facecolor="grey", alpha=0.3)
        ax.axvspan(t1.to_numpy(), t2.to_numpy(), facecolor="green", alpha=0.3)
        ax.plot(
            time.loc[t0:t1], reconstructed.loc[t0:t1, target], ls=":", lw=2, color="y"
        )
        ax.plot(
            time.loc[t1:t2], reconstructed.loc[t1:t2, target], ls="-", lw=2, color="r"
        )

### Create the Raw data plot

In [ ]:
%matplotlib widget

fig, axes = plt.subplots(
    nrows=3, ncols=3, sharex=True, figsize=(16, 8), constrained_layout=True
)

for ax, target in zip(axes.flatten(), task.targets):
    data = ts[target]
    times = ts.index.values
    mask = ~np.isnan(data)
    ax.plot(
        times[mask],
        data[mask],
        ls=":",
        lw=0.5,
        marker=".",
        ms=6,
    )
    ax.legend([f"{target} - observations"])

### Add the model Forecast Plots

In [ ]:
grid = np.linspace(0, len(samples) - 1, num=3, dtype=int)
# grid = [0]
batch = [samples[key] for key in grid]

for sample in batch:
    make_plot(axes, task, sample)

fig.savefig(f"{NAME.replace(r'/', r'_')}_{PRD_HORIZON}.pdf")

In [ ]:
raise

## Creating per-channel plots for all channels

In [ ]:
def make_plot(axes, task, batch, target):
    """Helper function to create plot automatically."""
    # batch = slices[0]
    times, inputs, targets, originals = (x.to(device="cpu") for x in prep_batch(batch))
    outputs = model(times, inputs)

    times = times[0].detach().cpu()
    inputs = inputs[0].detach().cpu()
    outputs = outputs[0].detach().cpu()
    targets = targets[0].detach().cpu()
    originals = originals[0].detach().cpu()

    times.shape, outputs.shape, inputs.shape, targets.shape, originals.shape
    reconstructed = encoder.decode((times, outputs)).astype("float32")

    data = originals[:, idx]
    mask = ~np.isnan(data)
    ax.plot(
        reconstructed.index[: task.observation_horizon],
        reconstructed.iloc[: task.observation_horizon, idx],
        ls=":",
        lw=0.1,
        color="r",
    )
    ax.plot(
        reconstructed.index[task.observation_horizon - 1 :],
        reconstructed.iloc[task.observation_horizon - 1 :, idx],
        ls="-",
        lw=2,
        color="r",
    )
    print(reconstructed)

In [ ]:
columns = task.dataset.timeseries.columns

In [ ]:
%matplotlib widget

fig, axes = plt.subplots(
    nrows=2, ncols=2, sharex=True, figsize=(16, 8), constrained_layout=True
)

for target in task.targets.items():
    data = ts[target]
    times = ts.index.values
    mask = ~np.isnan(data)
    ax.plot(
        times[mask],
        data[mask],
        ls=":",
        lw=0.5,
        marker=".",
        ms=6,
    )
    ax.legend([f"{target} - observations"])